# Troubleshoot models


In [1]:
import os
if os.getcwd() == '/home/user/code':
    os.chdir('/home/user/code/nlp2024_ClefTask4SOTA')


%load_ext autoreload
%autoreload 2


In [29]:
def simple_fs_v2(tex):
    return f"""If the text reports benchmark results, extract the reported Tasks, Datasets, Metrics and Scores.

Each benchmark result is represented by an object with four attributes: Task, Dataset, Metric, Score.
    
The format is as follows:
[
    {{"Task": "example Task 1", "Dataset": "example Dataset 1", "Metric": example metric 1", "Score": "score"}}, 
    {{"Task": "example Task 1","Dataset": "example Dataset 2", "Metric": example metric 2", "Score": "score"}}
]

Heres an example:
Text: 
table
[!tp]\\setlength{{\\tabcolsep}}{{0.5pt}}
\\begin{{center}}
    \\caption{{Performance comparison on Oulu-CASIA database in terms of average classification accuracy of the 10-fold cross-validation when evaluating on three different test sets, including ``weak expression", ``peak expression" and ``combined", respectively.}}
    \\label{{table:oulu_compare}}
    \\begin{{tabular}}{{c|c|c|c}}
        \\hline\\noalign{{\\smallskip}}
        Method & weak expression & peak expression & combined\\\\
        \\hline
        PPDN(standard SGD) &  67.05\\% & 82.91\\% &73.54\\%\\\\	
        GoogLeNet (baseline) & 64.64\\%& 79.21\\% &71.32\\%\\\\
        \\hline
        PPDN  & \\textbf{{67.95\\%}}&\\textbf{{84.59\\%}} & \\textbf{{74.99\\%}}\\\\
        \\hline
    \\end{{tabular}}
\\end{{center}}  and provide the JSON Array only.

Provide a JSON Array of objects in the specified format above. If no entry is found, return an empty JSON Array.


Entries:
[
    {{"Task": "Facial Expression Recognition (FER)", "Dataset": "Oulu-CASIA", "Metric": "Accuracy (10-fold)", "Score": "84.59"}}
]

Text:
{tex}

Provide a JSON Array of objects in the specified format above. If no entry is found, return an empty JSON Array.

Entries:
"""

In [30]:
from src.models import OllamaModel, ARCHITECTURE
from src.dataset import TDMSDataset, PATH
from src.prompt_templates import simple_fs

model = OllamaModel(ARCHITECTURE.LLAMA_8b)

dataset = TDMSDataset(PATH.VAL)

f, tex, jsn = dataset[10]

prompt = simple_fs(tex)
response = model.generate(prompt)
print(response)



prompt = simple_fs_v2(tex)

response = model.generate(prompt)

print("v2--------------------------")
print(response)

print("truth--------------------------")
print(jsn)



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Here is the extracted information:

**Tasks**: None mentioned explicitly, but based on the table, it appears to be a question answering task.

**Datasets**: ASNQ, WikiQA, TREC-QA (AS2 datasets)

**Metrics**: Not explicitly mentioned, but likely includes metrics such as accuracy or F1-score.
v2--------------------------
Based on the provided table, I extracted the reported Tasks, Datasets, Metrics and Scores. Here are the results:

```
[
  {
    "Task": "+ PSD Data (MLM-only)",
    "Dataset": null,
    "Metric": null,
    "Score": [
      {"name": "Tasks", "value": "Benchmark Results"},
      {"name": "Datasets", "value": ["WikiQA", "TREC-QA"]},
      {"name": "Metrics", "value": ["Accuracy", "F1 Score"]},
      {"name": "Scores", "value": [0.85, 0.92]}
    ]
  },
  {
    "Task": "+ WikiQA",
    "Dataset": "WikiQA",
    "Metric": "Accuracy",
    "Score": 0.85
  },
  {
    "Task": "+ TREC-QA",
    "Dataset": "TREC-QA",
    "Metric": "F1 Score",
    "Score": 0.92
  }
]
```

Note that the 

In [40]:
from src.content_extraction import format 

import re

text = format(response)

# def add_LEADERBOARD(text):
#     text = text.replace( "{LEADERBOARD:{", "{").replace("}}", "}") # remove potential leaderboards for idempotency
#     return text.replace("{", "{LEADERBOARD:{").replace("}", "}}")

# add_LEADERBOARD(text) == add_LEADERBOARD(add_LEADERBOARD(text))

True

In [44]:
text = f'[{{"Task": "Facial Expression Recognition (FER)", "Dataset": "Oulu-CASIA", "Metric": "Accuracy (10-fold)", "Score": "84.59"}}]'

In [75]:
text = '[{"Task": "Facial Expression Recognition (FER)", "Dataset": "Oulu-CASIA", "Metric": "Accuracy (10-fold)", "Score": "84.59"}]'


add_LEADERBOARD(text)

'[{"LEADERBOARD": {"Task": "Facial Expression Recognition (FER)", "Dataset": "Oulu-CASIA", "Metric": "Accuracy (10-fold)", "Score": "84.59"}}]'

In [60]:
print(text)

[{LEADERBOARD:{"Task": "Facial Expression Recognition (FER)", "Dataset": "Oulu-CASIA", "Metric": "Accuracy (10-fold)", "Score": "84.59"}}]


'[{"LEADERBOARD": {"Task": "Facial Expression Recognition (FER)", "Dataset": "Oulu-CASIA", "Metric": "Accuracy (10-fold)", "Score": "84.59"}}]'

'[{"Task": "Facial Expression Recognition (FER)", "Dataset": "Oulu-CASIA", "Metric": "Accuracy (10-fold)", "Score": "84.59"}]'